In [3]:
import Analysis
import cPickle as pickle
reload(Analysis)

A = Analysis.Analysis(tag='P7REP_CLEAN_V15_calore')

A.GenSquareMask(l_range=[-20.,20.], b_range=[-20.,20.], plane_mask=2.)
A.BinPhotons(infile='binned_photons_'+A.tag+'.npy')

# Load 2FGL 
A.AddPointSourceTemplate(fixNorm=True)

A.CalculatePixelWeights(diffuse_model='fermi_diffuse_'+A.tag+'.npy',psc_model='PSC_' + A.tag + '.npy',
                        alpha_psc=5., f_psc=0.05)

A.AddIsotropicTemplate(fixNorm=False, fixSpectrum=False) # External chi^2 used to fix normalization within uncertainties

A.AddFermiBubbleTemplate(template_file='./bubble_templates_diskcut30.0.fits', 
                         spec_file='./reduced_bubble_spec_apj_793_64.dat', fixSpectrum=False, fixNorm=False)

A.AddDMTemplate(profile='NFW', limits=[None,None], decay=False, gamma=1.25, 
               r_s=20.0, axesratio=1, offset=(0, 0), spec_file=None,)

A.PrintTemplates()

                NAME                    LIMITS      VALUE    FIXNORM    FIXSPEC   SRCCLASS
                 PSC              [None, None]  1.000e+00       True       True        PSC
             Bubbles              [None, None]  1.000e+00      False      False        GEN
                  DM              [None, None]  1.000e+00      False      False        GEN
           Isotropic              [None, None]  1.000e+00      False      False        ISO


In [8]:
models = ['base_2D', 'base','Pohl_HI_8500','Pohl_H2_8500', 'Pohl_HI_H2_8500']
models = ['base_2D','Pohl_HI_8500_L','Pohl_HI_8500_O','Pohl_HI_8500_S','Pohl_HI_8500_Y']
#models = ['Tavakoli_HI_L']
models = ['mod_b_'+ str(i) for i in range(33,33)]
#models = ['base_2D',]
# Load various diffuse models and run fits.
for galprop_tag in models:
    print galprop_tag
    A = Analysis.Analysis(tag='P7REP_CLEAN_V15_calore')
    A.GenSquareMask(l_range=[-20.,20.], b_range=[-20.,20.], plane_mask=2.)
    A.BinPhotons(infile='binned_photons_'+A.tag+'.npy')
    # Load 2FGL 
    A.AddPointSourceTemplate(fixNorm=True)
    A.CalculatePixelWeights(diffuse_model='fermi_diffuse_'+A.tag+'.npy',psc_model='PSC_' + A.tag + '.npy',
                        alpha_psc=5., f_psc=0.05)
    A.AddIsotropicTemplate(fixNorm=False, fixSpectrum=False) # External chi^2 used to fix normalization within uncertainties
    A.AddFermiBubbleTemplate(template_file='./bubble_templates_diskcut30.0.fits', 
                         spec_file='./reduced_bubble_spec_apj_793_64.dat', fixSpectrum=False, fixNorm=False)
    A.AddDMTemplate(profile='NFW', limits=[None,None], decay=False, gamma=1.25, 
               r_s=20.0, axesratio=1, offset=(0, 0), spec_file=None,)
    
    #A.ResetFit()
#     try:
#         A.AddGalpropTemplate('/data/galprop2/output/',galprop_tag,verbosity=1, multiplier=1., bremsfrac=1.25, 
#                       E_subsample=1, fixSpectrum=False)
#     except:
    A.AddHDF5Template(hdf5file=galprop_tag+'.hdf5',verbosity=1, multiplier=2., bremsfrac=1.25, 
                  E_subsample=2, fixSpectrum=False, separate_ics=False)
    A.RunLikelihood(print_level=1, tol=2e2, precision=None, minos=False)
    base = A.SaveFit(filename='./results/FitResult_'+ galprop_tag + '.pickle')




In [14]:
import pickle
plt.figure(figsize=(5,10))

#models = ['base_2D','Pohl_H2_8500_old_secondary']#,'Pohl_H2_8500']
#models = ['base_2D','Pohl_HI_8500_L','Pohl_HI_8500_O','Pohl_HI_8500_S','Pohl_HI_8500_Y']#,'Pohl_H2_8500']

#fits = [pickle.load(open('./results/FitResult_'+ galprop_tag + '.pickle','rb')) for galprop_tag in models]

models = ['mod_'+ str(i) for i in range(59)]
fits = []

for galprop_tag in models:
        try:
            mod = pickle.load(open('./results/FitResult_'+ galprop_tag + '.pickle','rb'))
            fits.append(mod)
        except:
            pass

plt.subplot(311)



for i in range(len(models)):
    try:
        fit = fits[i]
        print 'log-like, TS:', models[i], np.sum(fit['loglike']), 2*(np.sum(fits[0]['loglike'])-np.sum(fit['loglike']))

        E, flux, stat = fit['energies'], fit['DM']['flux'], fit['DM']['fluxunc']
        model = models[i].replace('_no_secondary','').replace('_','-')

        if i==0:
    #         plt.errorbar(E/1e3,E**2*flux/1e3,(E**2*stat[:,0]/1e3,E**2*stat[:,1]/1e3), 
    #                      label=model,capthick=1, capsize=2, ls=':' )
            plt.errorbar(E/1e3,E**2*flux/1e3,(E**2*stat[:]/1e3,E**2*stat[:]/1e3), 
                         label=model,capthick=1, capsize=2, ls=':' )
        else:
            plt.errorbar(E/1e3,E**2*flux/1e3, 
                         label=model,capthick=1, capsize=2, ls='-' )



        #E, flux, stat = fit['energies'], fit['Pi0_Brems']['flux'], fit['Pi0_Brems']['fluxunc']
        #model = models[i].replace('_no_secondary','').replace('_','-')

        if i==0:
    #         plt.errorbar(E/1e3,E**2*flux/1e3,(E**2*stat[:,0]/1e3,E**2*stat[:,1]/1e3), 
    #                      label=model,capthick=1, capsize=2, ls=':' )
            plt.errorbar(E/1e3,E**2*flux/1e3,(E**2*stat[:]/1e3,E**2*stat[:]/1e3), 
                         capthick=1, capsize=2, ls=':' )
        else:
            plt.errorbar(E/1e3,E**2*flux/1e3, 
                         capthick=1, capsize=2, ls='-' )
    except:
        pass
        
        
plt.legend(frameon=False, fontsize=8)
plt.yscale('log')
plt.xscale('log')
plt.ylabel(r'$\rm{E^2\ dN/dE\ [GeV/cm^2/s/sr]}$')
plt.xlabel(r'$\rm{E\ [GeV]}$')
plt.ylim(5e-8,4e-6)


E, f0, stat0 = fits[0]['energies'], fits[0]['DM']['flux'], fits[0]['DM']['fluxunc']

for i in range(1,len(models)):
    try:
        plt.subplot(312)
        fit = fits[i]
        E, flux, stat = fit['energies'], fit['DM']['flux'], fit['DM']['fluxunc']
        model = models[i].replace('_no_secondary','').replace('_','-')

        plt.errorbar(E/1e3,flux/f0, label=model,capthick=1, capsize=2, ls='-' )

        #plt.yscale('log')
        plt.xscale('log')
        plt.ylim(0,2)
        plt.ylabel(r'GCE Ratio')
        plt.xlabel(r'$\rm{E\ [GeV]}$')
        plt.legend(frameon=False, fontsize=8)

        plt.subplot(313)


        TS = -2*(np.array(fit['loglike'])-np.array(fits[0]['loglike']))
        plt.plot(E/1e3, TS)
        plt.ylabel('TS')
        #print "TS:", TS
        plt.xlabel(r'$\rm{E\ [GeV]}$')
        plt.xscale('log')
        plt.ylim(-200,200)
    except:
        pass
    

#plt.errorbar(E/1e3,np.ones(len(E)),yerr=(np.abs(stat0[:,0])/f0,stat0[:,1]/f0), linestyle='')

# plt.xscale('log')
# plt.ylabel(r'$\rm{Ratio\ of\ GCE\ 3FGL/2FGL}$')
# plt.xlabel(r'$\rm{E\ [GeV]}$')

# plt.savefig('/home/carlson/Dropbox/plots/GCE_sys/GCE_3fgl_vs_2fgl.pdf')

    

In [9]:


plt.figure(figsize=(5,6))

# 2FGL 
plt.subplot(211)

flux, stat = fgl2Fit['DM']['flux'], fgl2Fit['DM']['fluxunc']

E = fgl2Fit['energies']

plt.errorbar(E/1e3,E**2*flux/1e3,(E**2*stat[:,0]/1e3,E**2*stat[:,1]/1e3), 
             label='GCE-2FGL',capthick=1, capsize=2, ls=':' )

# 3FGL 
flux3, stat3 = fgl3Fit['DM']['flux'], fgl3Fit['DM']['fluxunc']
E = fgl3Fit['energies']
plt.errorbar(E/1e3,E**2*flux3/1e3,(E**2*stat3[:,0]/1e3,E**2*stat3[:,1]/1e3), 
             label='GCE-3FGL',capthick=1, capsize=2, ls=':' )


plt.legend()
plt.yscale('log')
plt.xscale('log')
plt.ylabel(r'$\rm{E^2\ dN/dE\ [GeV/cm^2/s/sr]}$')
plt.xlabel(r'$\rm{E\ [GeV]}$')

plt.subplot(212)


plt.errorbar(E/1e3,flux3/flux, 
             label='ratio',capthick=1, capsize=2, ls=':' )


plt.xscale('log')
plt.ylabel(r'$\rm{Ratio\ of\ GCE\ 3FGL/2FGL}$')
plt.xlabel(r'$\rm{E\ [GeV]}$')

plt.savefig('/home/carlson/Dropbox/plots/GCE_sys/GCE_3fgl_vs_2fgl.pdf')


In [197]:
#A.PrintTemplates()
import healpy
print 'E', A.central_energies[13]
m = healpy.cartview(A.psc_weights[13], return_projected_map=True,latra=[-20,20], lonra=[-20,20])
plt.clf()
plt.imshow(m,vmin=0,vmax=1)
plt.colorbar()
plt.show()

In [200]:
A.ResetFit()
#     try:
#         A.AddGalpropTemplate('/data/galprop2/output/',galprop_tag,verbosity=1, multiplier=1., bremsfrac=1.25, 
#                       E_subsample=1, fixSpectrum=False)
#     except:
A.AddHDF5Template(hdf5file='Pohl_HI_8500_L.hdf5',verbosity=1, multiplier=1., bremsfrac=1.25, 
              E_subsample=1, fixSpectrum=False, separate_ics=False)
A.RunLikelihood(print_level=1, tol=2e2, precision=None, minos=False)
#base = A.SaveFit(filename='./results/FitResult_'+ galprop_tag + '.pickle')

In [212]:
import healpy
from scipy.ndimage import gaussian_filter as gf
b = 10
m1 = healpy.cartview(A.residual[b],lonra=[-20,20],latra=[-20,20], return_projected_map=True, hold=True)
m2 = healpy.cartview(A2.residual[b],lonra=[-20,20],latra=[-20,20], return_projected_map=True, hold=True)
plt.clf()

print np.sum(A.loglike), np.sum(A2.loglike)

plt.imshow(gf(m1,5), vmin=-5,vmax=5, cmap='bwr', origin='lower')
plt.colorbar()
plt.show()
plt.imshow(gf(m2,5), vmin=-5,vmax=5, cmap='bwr', origin='lower')
plt.colorbar()
plt.show()

plt.imshow(gf(m1,5)/gf(m2,5), vmin=-5,vmax=5, cmap='bwr', origin='lower')
plt.colorbar()
plt.show()


In [199]:
import Analysis
import cPickle as pickle
reload(Analysis)

A2 = Analysis.Analysis(tag='P7REP_CLEAN_V15_calore')

A2.GenSquareMask(l_range=[-20.,20.], b_range=[-20.,20.], plane_mask=2.)
A2.BinPhotons(infile='binned_photons_'+A2.tag+'.npy')

# Load 2FGL 
A2.AddPointSourceTemplate(fixNorm=True)

A2.CalculatePixelWeights(diffuse_model='fermi_diffuse_'+A2.tag+'.npy',psc_model='PSC_' + A2.tag + '.npy',
                        alpha_psc=5., f_psc=0.1)

A2.AddIsotropicTemplate(fixNorm=False, fixSpectrum=False) # External chi^2 used to fix normalization within uncertainties

A2.AddFermiBubbleTemplate(template_file='./bubble_templates_diskcut30.0.fits', 
                         spec_file='./reduced_bubble_spec_apj_793_64.dat', fixSpectrum=False, fixNorm=False)

A2.AddDMTemplate(profile='NFW', limits=[None,None], decay=False, gamma=1.25, 
               r_s=20.0, axesratio=1, offset=(0, 0), spec_file=None,)

A2.PrintTemplates()

A2.AddHDF5Template(hdf5file='base_2D.hdf5',verbosity=1, multiplier=1., bremsfrac=1.25, 
              E_subsample=1, fixSpectrum=False, separate_ics=False)
A2.RunLikelihood(print_level=1, tol=2e2, precision=None, minos=False)

In [177]:
from scipy import ndimage
m = healpy.cartview(A.binned_data[10], latra=[-20,20], lonra=[-20,20], return_projected_map=True) 
plt.clf()
plt.imshow(np.log10(ndimage.gaussian_filter(m,1)), origin='lower',vmin=0)
plt.colorbar()



In [163]:
hdu = pyfits.open('/data/galprop2/FITS/CO_Pohl_8500_rlb.fits')
print hdu[0].shape
plt.figure(figsize=(6,6))
print hdu[0].data[:,(1441-160):(1441+160),:].shape


from scipy.ndimage.filters import gaussian_filter1d as gf
d = np.roll(hdu[0].data,1440,axis=1)[240-160:240+160,(1441-160):(1441+160),:]

d = gf(d,sigma=1 , mode='wrap', axis=1, truncate=10)
d = gf(d,sigma=1 , mode='nearest', axis=0, truncate=10)
d = np.clip(d, 1e-10,1e10)
plt.imshow(np.log10(np.sum(d,axis=2)), origin='lower', vmin=-1, vmax=3)
plt.xlim(320,0)
plt.colorbar()


In [164]:
hdu = pyfits.open('/data/galprop2/FITS/rbands_co10mm_v2_2001_qdeg_9R.fits')
print hdu[0].shape
plt.figure(figsize=(6,6))
print hdu[0].data[:,(1441-160):(1441+160),:].shape
plt.imshow(np.log10(np.sum(np.roll(hdu[0].data,720,axis=2)[:,(360-80):(360+80),(720-80):(720+80)],axis=0).clip(1e-10)), 
           origin='lower', vmin=-1,vmax=3)
plt.xlim(160,0)
plt.colorbar()